In [1]:
import ctypes
import sys

# Stacks

## Stack Implementations

In [2]:
class Empty(Exception):
    pass

In [3]:
class ArrayStack:
    '''LIFO Stack implementation using python list as an underlying storage.'''
    
    def __init__(self):
        '''Create an empty stack.'''
        self._data = []
    
    def __len__(self):
        '''Return the number of elements in the stack.'''
        return len(self._data)
    
    def push(self, e):
        '''Add an element to the top of the stack.'''
        self._data.append(e)
    
    def pop(self):
        '''Remove and return the element from the top of the stack.'''
        if self.is_empty():
            raise Empty('Stack underflow.')
        return self._data.pop()

    def top(self):
        '''Return the top element of the stack.'''
        if self.is_empty():
            raise Empty('Stack is empty.')
        return self._data[-1]
    
    def is_empty(self):
        '''Return True if the stack is empty.'''
        return len(self._data) == 0
    

In [4]:
class ResizedArrayStack:
    """LIFO Stack implementation using low level resized arrays."""

    def __init__(self, size=1):
        """Create an empty stack."""
        self._data = self._make_array(size)
        self._size = 0
        self._capacity = size

    def __len__(self):
        """Return the number of elements in the stack."""
        return self._size

    def push(self, e):
        """Add an element to the top of the stack."""
        if self._size == self._capacity:
            self._resize(self._capacity * 2)
        self._data[self._size] = e
        self._size += 1

    def pop(self):
        """Remove and return the element from the top of the stack."""
        if self.is_empty():
            raise Empty("Stack underflow.")
        self._size -= 1
        e = self._data[self._size]
        self._data[self._size] = None
        if 0 < self._size == self._capacity / 4:
            self._resize(self._capacity // 2)
        return e

    def top(self):
        """Return the top element of the stack."""
        if self.is_empty():
            raise Empty("Stack underflow.")
        return self._data[self._size - 1]

    def is_empty(self):
        """Return True if the stack is empty."""
        return self._size == 0

    def reverse(self):
        if self._size <= 1:
            return
        for i in range(self._size // 2):
            self._data[i], self._data[self._size - i - 1] = (
                self._data[self._size - i - 1],
                self._data[i],
            )

    def __iter__(self):
        """Return the class itself as an iterator."""
        self._position = self._size
        return self

    def __next__(self):
        """Returns the next element in the stack or raise StopIteration error."""
        self._position -= 1
        if self._position < 0:
            raise StopIteration()
        e = self._data[self._position]
        return e

    def _resize(self, capacity):
        """Resize internal array to new capacity c."""
        B = self._make_array(capacity)
        for i in range(self._size):
            B[i] = self._data[i]
        self._data = B
        self._capacity = capacity

    def _make_array(self, capacity):
        """Return an empty array with capacity c."""
        return (capacity * ctypes.py_object)()

In [5]:
S = ResizedArrayStack()
S.push(5)
S.push(3)

In [6]:
len(S)

2

In [7]:
S.top()

3

In [8]:
S.pop()

3

In [9]:
S.top()

5

In [10]:
len(S)

1

In [11]:
for s in S:
    print(s)

5


In [12]:
S.push(10)
S.push(100)

In [13]:
for s in S:
    print(s)

100
10
5


In [14]:
vars(S)

{'_data': <__main__.py_object_Array_4 at 0x7fba39879c20>,
 '_size': 3,
 '_capacity': 4,
 '_position': -1}

In [15]:
S.reverse()

In [16]:
for s in S:
    print(s)

5
10
100


## Stack Applications

### Reversing lines in a file

In [17]:
with open('file.txt', 'r') as f:
    for line in f.readlines():
        print(line.rstrip())

Line1
Line2
Line3


In [18]:
S = ResizedArrayStack()
with open('file.txt', 'r') as f:
    for line in f.readlines():
        S.push(line.rstrip('\n'))

In [19]:
for line in S:
    print(line)

Line3
Line2
Line1


In [20]:
with open('reversed-file.txt', 'w') as f:
    while not S.is_empty():
        f.write(S.pop() + '\n')

In [21]:
with open('reversed-file.txt', 'r') as f:
    for line in f.readlines():
        print(line.rstrip())

Line3
Line2
Line1


### Matching Parentheses

In [22]:
def is_matched(expr):
    left = '({['
    right = ')}]'
    S = ResizedArrayStack()
    for char in expr:
        if char in left:
            S.push(char)
        elif char in right:
            if S.is_empty():
                return False
            if right.index(char) != left.index(S.pop()):
                return False
    return S.is_empty()

In [23]:
is_matched('()')

True

In [24]:
is_matched('({[]})')

True

In [25]:
is_matched('(')

False

In [26]:
is_matched('(()')

False